In [127]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time
import logging
from tqdm import tqdm

# Visulatisation
%matplotlib widget
import matplotlib.pyplot as plt

import set_path
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.plot_utils as plot_utils
import src.utils.clip_utils as clip_utils
import src.utils.math_utils as math_utils
import src.utils.bgt_utils as bgt_utils

from src.utils.interpolation import FastGridInterpolator

from pyntcloud import PyntCloud
import pandas as pd
from skimage import feature
from scipy import ndimage
import cv2 as cv
import pyransac3d as pyrsc

import open3d as o3d
from shapely.geometry import LineString
import shapely 
from scipy.optimize import curve_fit
import scipy.spatial

from multiprocessing import Pool
from sklearn.cluster import DBSCAN

from skimage.transform import probabilistic_hough_line, hough_line, hough_line_peaks

In [164]:
# Load data.
tilecode = '2387_9700'

# Labelled LAS file (see Notebook 0).
las_file = '../datasets/Valeriusplein/pointcloud/processed_' + tilecode + '.laz'
reduced_las_file = '../datasets/Valeriusplein/pointcloud/M2_' + tilecode + '.laz'

# AHN data folder.
ahn_data_folder = '../datasets/Valeriusplein/ahn/'
# File with BGT building polygons.
bgt_building_file = '../datasets/Valeriusplein/bgt/bgt_buildings.csv'
# File with BGT road polygons.
bgt_road_file = '../datasets/Valeriusplein/bgt/bgt_roads.csv'
# File with <x,y> coordinates of pole-like objects.
bgt_pole_file = '../datasets/Valeriusplein/bgt/custom_poles.csv'
# File with <x,y> coordinates of street furniture objects.
bgt_street_furniture_file = '../datasets/Valeriusplein/bgt/bgt_street_furniture.csv'
# File with tram tracks linestrings.
bgt_tram_track_file = '../datasets/Valeriusplein/bgt/bgt_tram_tracks.csv'

# AHNReader for elevation data.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [169]:
plot_utils.plot_bgt(tilecode, bgt_building_file, bgt_road_file, bgt_pole_file, bgt_street_furniture_file, bgt_tram_track_file)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:
# true pointcloud
labeled_pointcloud = las_utils.read_las(las_file)
true_labels = labeled_pointcloud.label

# processed pointcloud
pointcloud = las_utils.read_las(reduced_las_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

# mask for cable points
mask = np.where(labels == 11)[0]

In [95]:
def plot_clusters(points, clusters_labels, tilecode, exclude_clusters=(), plot_3d=True, tram_tracks=None, padding=2.5):
    clusters = set(clusters_labels).difference(exclude_clusters)
    fig = plt.figure()

    if plot_3d:
        ax = plt.axes(projection='3d')
        for cluster in clusters:
            c_mask = clusters_labels == cluster
            ax.scatter(points[c_mask][:,0],points[c_mask][:,1],points[c_mask][:,2], s=2, label=str(cluster))
    else:
        ax = plt.axes()
        for cluster in clusters:
            c_mask = clusters_labels == cluster
            ax.scatter(points[c_mask][:,0],points[c_mask][:,1], s=1,label=str(cluster))
            
    plt.legend()
    plt.show()

In [186]:
def tramtrack_polygon(tilecode, bgt_tram_track_file):

    # Read tram tracks
    tram_tracks = bgt_utils.get_linestrings(bgt_tram_track_file, tilecode, padding=10)    
    
    # Read tile bbox
    bbox = las_utils.get_bbox_from_tile_code(tilecode)
    tile_poly = shapely.geometry.Polygon([bbox[0],(bbox[0][0]+50, bbox[0][1]), bbox[1],(bbox[0][0], bbox[0][1]-50)])

    # Create tramtrack polygon
    tramtracks_poly = shapely.geometry.Polygon()
    for track in tram_tracks:
        if tile_poly.intersects(track):
            track_in_tile = tile_poly.intersection(track)
            tramtracks_poly = tramtracks_poly.union(track_in_tile)

    # Grow and erode tramtrack polygon
    tramtracks_poly = tramtracks_poly.buffer(2.5).buffer(-1).intersection(tile_poly)
    return tramtracks_poly

In [189]:
def cable_classification(points, mask, plot=True):

    # AHN ground interpolation
    ahn_tile = ahn_reader.filter_tile(tilecode)
    values = ahn_tile['ground_surface']
    fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'], values)

    # Tramtrack area polygon
    tramtracks_poly = tramtrack_polygon(tilecode, bgt_tram_track_file)

    # Cluster cable points
    clustering = (DBSCAN(eps=1, min_samples=4, p=2).fit(points[mask]))
    #plot_clusters(points[mask], clustering.labels_, tilecode, plot_3d=True)

    # Calculate cable height
    cable_labels = np.zeros(len(mask))
    for cc in set(clustering.labels_):
        cc_mask = clustering.labels_ == cc
        cc_pts = points[mask][cc_mask]

        # A: cable location
        cbbox = math_utils.compute_bounding_box(cc_pts[:,:2])
        cable_poly = shapely.geometry.Polygon([cbbox[:2], (cbbox[0], cbbox[3]), cbbox[2:], (cbbox[2], cbbox[1])])

        # cable_mbr = math_utils.minimum_bounding_rectangle(cc_pts[:,:2])[0]
        # cable_poly = shapely.geometry.Polygon(cable_mbr)
        crosses_tramtracks = cable_poly.intersects(tramtracks_poly)
        
        # B: cable height
        cc_height =  cc_pts[:, 2] - fast_z(cc_pts)

        # Rule based classification
        # 0: undefined
        # 1: tram cable
        # 2: light cable
        if crosses_tramtracks:
            if cc_height.min() < 7.5:
                cable_labels[cc_mask] = 1
                # option to distinguish suspension vs power cable using direction?
            else:
                cable_labels[cc_mask] = 2
        else:
            cable_labels[cc_mask] = 2

    
    if plot:
        plot_clusters(points[mask], cable_labels, tilecode, plot_3d=True)

    return cable_labels

In [190]:
cable_classification(points, mask)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([2., 2., 2., ..., 2., 2., 2.])